In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder,LabelBinarizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score,precision_recall_fscore_support,f1_score
from sklearn.metrics import classification_report
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack
from pandas.tools.plotting import parallel_coordinates
from math import pi

wine_base = pd.read_csv("Our_dataset/StemmedWord2vecTop3_parsed.csv", index_col=0) 
print(wine_base.columns)
wine_base = wine_base.reset_index()
wine_base= wine_base[pd.notnull(wine_base['description'])]
wine_base.columns

for col in wine_base.columns:
    if(col.startswith("tf")):
            wine_base.drop(col,inplace=True,axis=1)

import re
def get_vintage(df):
    pattern = r"\d{4}"
    vintages = []
    for elem in df.title:
        match = re.findall(pattern,elem)
        year = 0
        if len(match)>1:
            y= int(match[0])
            if y <1952:
                year = 0
            elif y==3000:
                year == 2009
            elif y == 7200:
                year = int(match[1])
            else:
                year= y
        elif len(match)==1:
            y= int(match[0])
            if y <1952:
                year = 0
            elif y==3000:
                year == 2009
            elif year == 7200:
                year = 0
            else:
                year = y
        else:
            year = 0
        vintages.append(year)

    se = pd.Series(vintages)
    df['vintage']=se.values 
    df.drop('title',axis=1)
    return df

wine_base = get_vintage(wine_base)
#drop reviwe of wine with ventage =0
wine_base = wine_base[wine_base.vintage != 0]

wine_base = wine_base[pd.notnull(wine_base['country'])]
wine_base = wine_base[pd.notnull(wine_base['taster_name'])]
wine_base = wine_base[pd.notnull(wine_base["variety"])]
wine_base = wine_base[pd.notnull(wine_base["province"])]
wine_base = wine_base[pd.notnull(wine_base["winery"])]

wine_base.drop("title",inplace=True,axis=1)
wine_base = wine_base.dropna()

#keep track of the countries,provinces,taster_names in the database
countries = wine_base["country"].unique().tolist()
provinces = wine_base["province"].unique().tolist()
tasters = wine_base["taster_name"].unique().tolist()

def OneHotEncode(dataframe,column_to_encode,take_whole_dataset=True):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(dataframe[[column_to_encode]])
    mapping = {}
    i=0
    for elem in enc.categories_[0]:
        mapping[elem]=i
        i+=1
    resu = enc.transform(dataframe[[column_to_encode]]).toarray()
    if take_whole_dataset:
        for elem in mapping:
            dataframe[elem]=resu[:,mapping[elem]]
        return dataframe
    else:
        subset = dataframe[[column_to_encode]]
        for elem in mapping:
            subset[elem]=resu[:,mapping[elem]]
        return subset
    
wine_base = OneHotEncode(wine_base,'taster_name')
wine_base = wine_base.drop('taster_name', 1)
#not using regional stuff
#wine_base = OneHotEncode(wine_base,'province')
wine_base = wine_base.drop('province', 1)
wine_base.drop(["designation","description"],axis=1,inplace=True)

lb = LabelBinarizer()

#wine_base["winery"] = wine_base["winery"].astype('category').cat.codes
wine_base.drop("winery",axis=1,inplace=True)
#wine_base["region_1"] = wine_base["region_1"].astype('category').cat.codes
wine_base.drop("region_1",axis=1,inplace=True)
wine_base["variety"] = wine_base["variety"].astype('category').cat.codes
wine_base

In [ ]:
fig, ax = plt.subplots(figsize = (12, 5))
sns.distplot(wine_base["points"],hist=True,bins = 20,hist_kws={'edgecolor':'black'})

In [ ]:
countries = wine_base["country"].unique().tolist()
fig, ax = plt.subplots(figsize = (14, 6))
for c in countries:
    temp = wine_base[wine_base["country"] == c]
    if c != "Austria" and c!="Germany":
        plt.title("Points distribution for all countries")
        ax.set(xticks=wine_base["points"].unique())
        sns.distplot(temp["points"],label=c,hist=False,color="gray")
    else:
        sns.distplot(temp["points"],label=c,hist=False)
        plt.legend()

sns

Here you can see the difference in the distributions, all countries peak around 86/87/88, while germany and austria around 90. This could be why training on them and testing on the others sucks (or vice versa)

In [ ]:
countries = wine_base["country"].unique().tolist()
for c in countries:
    fig, ax = plt.subplots(figsize = (10, 5))
    temp = wine_base[wine_base["country"] == c]
    plt.title("Price distribution for " + c)
    sns.distplot(temp["price"],hist=True)
    sns

# DEFINE NUMBER OF BINS = classes to be predicted (must be executed so that Y is the same for every execution)

In [ ]:
Y = wine_base['points'].copy()
#DECIDE NUMBER OF BINS 
#nbins  = 4
#labels=["low","medium","high","very_high"]
#nbins = 5
#labels=["very_low", "low", "medium","high","very_high"]
#bin identici 
#Y,bins = pd.cut(Y,nbins,labels=labels,retbins=True,include_lowest=True,right=True)
#quartile
nbins  = 3
labels=["low","medium","high"]
Y,bins = pd.qcut(Y,nbins,labels=labels,retbins=True)   #uses quartiles and statistic stuff
values = Y.tolist()  

#plot the binning result
fig, ax = plt.subplots(figsize = (12, 5))
for i in range(1,len(bins)-1):

    if i == 1:
        a = wine_base[wine_base["points"] <= bins[i]]
        n =  bins[i]-80
        sns.distplot(a["points"],hist_kws={"width": 0.5,'edgecolor':'black'},kde=False)
        g =+n
    if i == len(bins)-1:
        a = wine_base[wine_base["points"] > bins[i]]
        n = 100 - bins[i]
        sns.distplot(a["points"],hist_kws={"width": 0.5,'edgecolor':'black'},kde=False)
        g =+ n
    else:
        n = bins[i+1] - bins[i]
        g =+n
        a = wine_base[(wine_base["points"] > bins[i]) & (wine_base["points"] <= bins[i+1])]
        sns.distplot(a["points"],hist_kws={"width": 0.5,'edgecolor':'black'},kde=False)
ax.set(xticks=wine_base["points"].unique())
plt.ylabel("Number of entries")
plt.xlabel("Points assigned to wines")
plt.title("Binning results")
print(g)
sns

# test on one country, train on the others

In [ ]:
countries = wine_base["country"].unique().tolist()
countries
for c in countries:
    print(c,len(wine_base[wine_base["country"] == c]))
test = wine_base["country"].unique().tolist()

In [ ]:
def testByCountry(data,countries):
    #for every country, use it as test set
    nbins  = 3
    labels=["low","medium","high"]
    wine2 = pd.DataFrame()
    wine2 = data
    g,b = pd.qcut(data["points"],nbins,labels=labels,retbins=True)   #uses quartiles and statistic stuff
    g.tolist()
    wine3 = wine2.drop(["points"],axis=1)
    wine3["category"] = g
    df = pd.DataFrame()
    for el in countries:
        res = applyTest(wine3,el)
        df = df.append(res)
    return df

def applyTest(data,objCountry):
  
    #set many different combinations
    #only on these
    allfeatures = []
    args = []
    for el in wine_base.columns:
        if el != "description":
            allfeatures.append(el)   
        if el != "description" and el != "points" and el != "country":
            args.append(el)
    combos = {"depth":[2,3,4,5,6,7,8],"args":[args]}
    #
    res = buildCountryResult(data,args,allfeatures,combos,objCountry)
    return res

def buildCountryResult(data,args,allfeatures,combination,objCountry):
    cols = ["algorithm","ObjectiveCountry","input","precision","accuracy","f1","depth"]
    algorithm = "decTree"
    for lab in labels:
        cols.append(lab +"_prec")
        cols.append(lab +"_recall")
        cols.append(lab +"_f1")
        cols.append(lab +"_support")
    for el in args: #controlla che allfeats vada bene
        c = "feat_"+el
        if c not in cols:
            cols.append(c)
    results = pd.DataFrame()
    comb = 0
    row = 0
    for el in combination["args"]: 
        el = el + ["country","category"]
        X = data[el]
        #create test and training sets
        X_train = X[X["country"] != objCountry]
        y_train = pd.DataFrame()
        #obbiettivo è la categoria del dataframe X_train perchè è quello che contiene le righe giuste per le country
        y_train = list(X_train["category"])
        print(objCountry +" must not be in training set " + str(X_train["country"].unique().tolist()))
        X_train.drop(["country","category"],axis=1,inplace=True)

        X_test = X[X["country"] == objCountry]
        y_test = X_test["category"]
        X_test.drop(["country","category"],axis=1,inplace=True)
        for depth in combination["depth"]:
                row = row + 1  
                #train stuff
                classifier = DecisionTreeClassifier(max_depth=depth)
                classifier.fit(X_train,y_train)
                y_pred = classifier.predict(X_test)  
                acc = accuracy_score(y_test,y_pred)#average="macro")
                weightedPrec = precision_score(y_test,y_pred,average="weighted")
                f1 = f1_score(y_test,y_pred,average="weighted")
                rowData = [algorithm,objCountry,el,weightedPrec,acc,f1,depth]
                precision,recall,fscore,support = precision_recall_fscore_support(y_test, y_pred)
                for i in range(0,len(labels)):
                    rowData.append(precision[i])
                    rowData.append(recall[i])
                    rowData.append(fscore[i])
                    rowData.append(support[i])
                    temp = {}
                c = zip(el,classifier.feature_importances_)
                names = []
                for name,importance in c:
                    temp[name] = importance
                    names.append(name)
                for feat in names:
                    #if feat not in allfeatures:
                     #   rowData.append(100) #100 is an impossible value not to be taken into account
                    #else:
                        rowData.append(temp[feat]) #qualcosa non quadra con le liste di input e colonne per appendere i dataframe con le relative feature importances
                df2 = pd.DataFrame([rowData],columns=cols)
                results = results.append(df2,ignore_index=True)
        comb = comb + 1
    return results

In [ ]:
basic = ["price",'vintage', 'province', 'region_1', 'taster_name', 'variety','winery'] #country non deve esserci 
word = ["word_count"]
word2vec = ['similarityTop3WinesByVariety']
features = []
for el in wine_base.columns:
    if el != "description":
        features.append(el)

result = testByCountry(wine_base,countries)
len(result.columns.tolist())
result.drop("input",inplace=True,axis=1)
result

In [ ]:
#if the column is never used delete it
for col in result.columns:
    if len(result[col].unique().tolist()) == 1 and col != "algorithm":
        result.drop(col,inplace=True,axis=1)
result.columns

In [ ]:
mean = result.groupby("ObjectiveCountry")["precision","accuracy","f1","low_f1","medium_f1","high_f1"].mean()
mean.reset_index(inplace=True)
mean

In [ ]:
#plot accuracy, precision, f1 for every country (f1=weighted average of precision, recall)
temp = mean.sort_values(["f1"],ascending=False)
# width of the bars
barWidth = 0.25
 
# Choose the height of the blue bars
bars1 = temp["precision"]
# Choose the height of the cyan bars
bars2 = temp["accuracy"]

bars3 = temp["f1"]

# The x position of bars
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
fig, ax = plt.subplots(figsize = (14, 5))

# Create blue bars
plt.bar(r1, bars1, width = barWidth, color = "#b2df8a", edgecolor = 'black', capsize=7, label='precision')
 
# Create cyan bars
plt.bar(r2, bars2, width = barWidth, color = '#1f78b4', edgecolor = 'black', capsize=7, label='accuracy')

plt.bar(r3, bars3, width = barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7, label='f1')

xs = temp["ObjectiveCountry"].unique().tolist()
# general layout
plt.xticks([r + barWidth for r in range(len(bars1))], xs)
plt.title("Results by testing on each country")
plt.ylabel('%')
plt.xlabel("Countries")
plt.legend()
 
# Show graphic
plt.show()


probably new zealand, italy and US are the least accurate testing because they're theh 

In [ ]:
#plot f1 by category predicted 
temp = mean.sort_values(["medium_f1"],ascending=False)

# width of the bars
barWidth = 0.25
 
# Choose the height of the blue bars
bars1 = temp["low_f1"]
# Choose the height of the cyan bars
bars2 = temp["medium_f1"]

bars3 = temp["high_f1"]

# The x position of bars
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
fig, ax = plt.subplots(figsize = (13, 5))

# Create blue bars
begin = plt.bar(r1, bars1, width = barWidth, color = "#fee6ce", edgecolor = 'black', capsize=7, label='low')
 
# Create cyan bars
plt.bar(r2, bars2, width = barWidth, color = '#fdae6b', edgecolor = 'black', capsize=7, label='medium')

end = plt.bar(r3, bars3, width = barWidth, color = "#e6550d", edgecolor = 'black', capsize=7, label='high')
xs = temp["ObjectiveCountry"].unique().tolist()
# general layout
plt.xticks([r + barWidth for r in range(len(bars1))], xs)
plt.ylabel('%')
plt.xlabel("Countries")
plt.legend()
plt.title("f1 by class, by testing model on the single country") 
# Show graphic
plt.show()

In [ ]:
#take mean of all the features used in the computations of decision tree to see the by country and understand which factors play into a country 
features = []
for col in df.columns:
    if col.startswith("feat"):
        features.append(col)
print(features)

t1 = pd.DataFrame()
t = result.groupby("ObjectiveCountry")
for col in features: 
    t1[col] = t[col].mean() 
t1.reset_index(inplace=True)
t1

In [ ]:
for idx,row in t1.iterrows():
    both = []
    for col in t1.columns:
        if row[col] != 0 and col != "ObjectiveCountry": #and col != "feat_price":    gnore price
            both.append([col,row[col]])
    #print(toPlot)
    #print(values)
    
    from operator import itemgetter
    both = sorted(both, key=itemgetter(1))
    print(both)
    
    toPlot = []
    values = []    
    for el in both:
        toPlot.append(el[0])
        values.append(el[1])
    
    # width of the bars
    barWidth = 0.5
    
    # Choose the height of the blue bars
    bars1 = values

    # The x position of bars
    r1 = np.arange(len(toPlot))
    fig, ax = plt.subplots(figsize = (10, 7))

    # Create blue bars
    plt.barh(r1, bars1, height=barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7)
    #plt.bar(r1,bars1,width = barWidth..)
    ys = toPlot
    # general layout
    #plt.xticks([r for r in range(len(bars1))], ys,rotation='vertical')
    #plt.yticks([r for r in range(len(bars1))], ys,rotation='vertical')
    plt.yticks([r for r in range(len(bars1))], ys)
    plt.title("Target country: "+ t1.iloc[idx]["ObjectiveCountry"])
    plt.ylabel('Features (except price)')
    plt.xlabel("Importance")
    # Show graphic
    plt.show()

# Train on one country, test on others

In [ ]:
#wine_base["country_tag"] = wine_base["country"].astype('category').cat.codes
#wine_base.drop("region_1",inplace=True,axis=1)

#this way, the only missing attribute is the country, so that the classifier doesn't work on which country it comes from

In [ ]:
def trainByCountry(data,countries):
    #for every country, use it as test set
    nbins  = 3
    labels=["low","medium","high"]
    wine2 = pd.DataFrame()
    wine2 = data
    g,b = pd.qcut(data["points"],nbins,labels=labels,retbins=True)   #uses quartiles and statistic stuff
    g.tolist()
    wine3 = wine2.drop(["points"],axis=1)
    wine3["category"] = g
    df = pd.DataFrame()
    for el in countries:
        res = applyTest1(wine3,el)
        df = df.append(res)
    return df

def applyTest1(data,objCountry):
  
    #set many different combinations
    #only on these
    allfeatures = []
    args = []
    for el in wine_base.columns:
        if el != "description":
            allfeatures.append(el)   
        if el != "description" and el != "points" and el != "country":
            args.append(el)
   # print("ARGS " + str(args))
    combos = {"depth":[2,3,4,5,6],"args":[args]}
    #
    res = buildCountryResult1(data,args,allfeatures,combos,objCountry)
    return res

def buildCountryResult1(data,args,allfeatures,combination,objCountry):
    cols = ["algorithm","ObjectiveCountry","input","precision","accuracy","f1","depth"]
    algorithm = "decTree"
    for lab in labels:
        cols.append(lab +"_prec")
        cols.append(lab +"_recall")
        cols.append(lab +"_f1")
        cols.append(lab +"_support")
    for el in args: #controlla che allfeats vada bene
        c = "feat_"+el
        if c not in cols:
            cols.append(c)
    results = pd.DataFrame()
    comb = 0
    row = 0
    #print(cols)
    for el in combination["args"]: 
        el = el + ["country","category"]
        X = data[el]
        #create test and training sets
        X_train = X[X["country"] == objCountry]
        y_train = pd.DataFrame()
        y_train = list(X_train["category"])
        #print(objCountry +" must be the only one in the training set " + str(X_train["country"].unique().tolist()))
        #print(str(X_train["country"].unique().tolist()) + "country in the training set")
        X_train.drop(["country","category"],axis=1,inplace=True)

        X_test = X[X["country"] != objCountry]
        y_test = X_test["category"]
       # print(str(X_test["country"].unique().tolist()) + "country in the test set")
        X_test.drop(["country","category"],axis=1,inplace=True)
        for depth in combination["depth"]:
                row = row + 1  
                #train stuff
                classifier = DecisionTreeClassifier(max_depth=depth)
                classifier.fit(X_train,y_train)
                y_pred = classifier.predict(X_test)  
                acc = accuracy_score(y_test,y_pred)#average="macro")
                weightedPrec = precision_score(y_test,y_pred,average="weighted")
                f1 = f1_score(y_test,y_pred,average="weighted")
                rowData = [algorithm,objCountry,el,weightedPrec,acc,f1,depth]
                precision,recall,fscore,support = precision_recall_fscore_support(y_test, y_pred)
                for i in range(0,len(labels)):
                    rowData.append(precision[i])
                    rowData.append(recall[i])
                    rowData.append(fscore[i])
                    rowData.append(support[i])
                    temp = {}
                c = zip(el,classifier.feature_importances_)
                names = []
                for name,importance in c:
                    temp[name] = importance
                    names.append(name)
                for feat in names:
                    #if feat not in allfeatures:
                     #   rowData.append(100) #100 is an impossible value not to be taken into account
                    #else:
                        rowData.append(temp[feat]) #qualcosa non quadra con le liste di input e colonne per appendere i dataframe con le relative feature importances
                df2 = pd.DataFrame([rowData],columns=cols)
                results = results.append(df2,ignore_index=True)
        comb = comb + 1
    return results

In [ ]:
trainedOnSingleCountry = trainByCountry(wine_base,countries)
trainedOnSingleCountry.drop("input",inplace=True,axis=1)
trainedOnSingleCountry

In [ ]:
#plot accuracy, precision, f1 for every country (f1=weighted average of precision, recall)
temp = pd.DataFrame()
temp = df.loc[:,["ObjectiveCountry","precision","accuracy","f1"]]
temp = temp.sort_values(["f1"],ascending=False)
# width of the bars
barWidth = 0.25
 
# Choose the height of the blue bars
bars1 = temp["precision"]
# Choose the height of the cyan bars
bars2 = temp["accuracy"]

bars3 = temp["f1"]

# The x position of bars
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
fig, ax = plt.subplots(figsize = (14, 5))

# Create blue bars
plt.bar(r1, bars1, width = barWidth, color = "#b2df8a", edgecolor = 'black', capsize=7, label='precision')
 
# Create cyan bars
plt.bar(r2, bars2, width = barWidth, color = '#1f78b4', edgecolor = 'black', capsize=7, label='accuracy')

plt.bar(r3, bars3, width = barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7, label='f1')

xs = temp["ObjectiveCountry"].unique().tolist()
# general layout
plt.xticks([r + barWidth for r in range(len(bars1))], xs)
plt.title("Results obtained by training on a single country and testing on the others")
plt.ylabel('%')
plt.xlabel("Countries")
plt.legend()
 
# Show graphic
plt.show()


In [ ]:

#plot f1 by category predicted 
temp = pd.DataFrame()
temp = df.loc[:,["ObjectiveCountry","low_f1","medium_f1","high_f1"]]
temp = temp.sort_values(["medium_f1","high_f1","low_f1"],ascending=False)
temp

# width of the bars
barWidth = 0.25
 
# Choose the height of the blue bars
bars1 = temp["low_f1"]
# Choose the height of the cyan bars
bars2 = temp["medium_f1"]

bars3 = temp["high_f1"]

# The x position of bars
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
fig, ax = plt.subplots(figsize = (14, 5))

# Create blue bars
plt.bar(r1, bars1, width = barWidth, color = "#fee6ce", edgecolor = 'black', capsize=7, label='low')
 
# Create cyan bars
plt.bar(r2, bars2, width = barWidth, color = '#fdae6b', edgecolor = 'black', capsize=7, label='medium')

plt.bar(r3, bars3, width = barWidth, color = "#e6550d", edgecolor = 'black', capsize=7, label='high')

#ys = temp["ObjectiveCountry"].unique().tolist()
# general layout
plt.xticks([r + barWidth for r in range(len(bars1))], xs) #xs definito sopra
plt.ylabel('%')
plt.xlabel("Countries")
plt.legend()
plt.title("Results (f1) obtained by training on a single country and testing on all the others") 
# Show graphic
plt.show()

In [ ]:
#take mean of all the features used in the computations of decision tree to see the by country and understand which factors play into a country 

features = []
for col in df.columns:
    if col.startswith("feat"):
        features.append(col)
print(features)

t1 = pd.DataFrame()
t = trainedOnSingleCountry.groupby("ObjectiveCountry")
for col in features: 
    t1[col] = t[col].mean() 
t1.reset_index(inplace=True)
t1

In [ ]:
for idx,row in t1.iterrows():
    both = []
    for col in t1.columns:
        if row[col] != 0 and col != "ObjectiveCountry": #THIS TAKES INTO ACCOUNT EVEN PRICE WHICH IS A LOT LESS IMPORTANT COMPARED TO THE FIRST APPROACH
            both.append([col,row[col]])
    #print(toPlot)
    #print(values)
    
    from operator import itemgetter
    both = sorted(both, key=itemgetter(1))
    print(both)
    
    toPlot = []
    values = []    
    for el in both:
        toPlot.append(el[0])
        values.append(el[1])
    
    # width of the bars
    barWidth = 0.5
    
    # Choose the height of the blue bars
    bars1 = values

    # The x position of bars
    r1 = np.arange(len(toPlot))
    fig, ax = plt.subplots(figsize = (10, 7))

    # Create blue bars
    plt.barh(r1, bars1, height=barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7)
    #plt.bar(r1,bars1,width = barWidth..)
    ys = toPlot
    # general layout
    #plt.xticks([r for r in range(len(bars1))], ys,rotation='vertical')
    #plt.yticks([r for r in range(len(bars1))], ys,rotation='vertical')
    plt.yticks([r for r in range(len(bars1))], ys)
    plt.title("Country on which model has been trained: "+ t1.iloc[idx]["ObjectiveCountry"])
    plt.ylabel('Features')
    plt.xlabel("Importance")
    # Show graphic
    plt.show()

# DUMMY (train one country, test others)

In [ ]:
for c in countries:
    X = wine_base
    wine_base["category"] = Y
    test_size = 0.30
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=42)
    X_train = X[X["country"] == c]
    y_train = list(X_train["category"])
    print(c +" must be the only one in the training set " + str(X_train["country"].unique().tolist()))
    print(str(X_train["country"].unique().tolist()) + "country in the training set")
    X_train.drop(["country","category"],axis=1,inplace=True)

    X_test = X[X["country"] != c]
    y_test = X_test["category"]
    print(str(X_test["country"].unique().tolist()) + "country in the test set")
    X_test.drop(["country","category"],axis=1,inplace=True)
   
    classifier = DummyClassifier("stratified")
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)  
    acc = accuracy_score(y_test,y_pred)
    weightedPrec = precision_score(y_test,y_pred,average="weighted")
    print("COUNTRY " + c + classification_report(y_test, y_pred, target_names=labels))

    

# DUMMY (country x country)


In [ ]:
cols = ["trainedOn","testedOn","f1"]

l = [None] * len(cols)
dummy = pd.DataFrame([l],columns=cols)
for c in countries:
    for c1 in countries:
        if c != c1:
            rowData = [c,c1]
            X = wine_base
            wine_base["category"] = Y
            X_train = X[X["country"] == c]
            y_train = list(X_train["category"])
            X_train.drop(["country","category"],axis=1,inplace=True)

            X_test = X[X["country"] == c1]
            y_test = X_test["category"]
            X_test.drop(["country","category"],axis=1,inplace=True)

            classifier = DummyClassifier("most_frequent")
            classifier.fit(X_train,y_train)
            y_pred = classifier.predict(X_test)  
            f1 = f1_score(y_test,y_pred,average="weighted")
            rowData.append(f1)
            df2 = pd.DataFrame([rowData],columns=cols)
            dummy = dummy.append(df2)

In [ ]:
dummy.reset_index(inplace=True,drop=True)
dummy.sort_values("f1",ascending=False)
dummy.dropna(inplace=True)
dummy

In [ ]:
temp1 = pd.DataFrame()
feats=["trainedOn","testedOn","f1"]
for f in feats:
    temp1[f] = dummy[f]
temp1 = temp1.sort_values("trainedOn",ascending=True)
temp1.reset_index(inplace=True,drop=True)
#t2 =pd.DataFrame()
#for col in feats:
#    if col != "combination" and col != "trainedOn" and col != "testedOn":
#        t2[col] = temp1[col]
#t2.reset_index(inplace=True)
#t2.sort_values("f1",ascending=False)
columns = ["trainedOn"]
countries = temp1["trainedOn"].unique().tolist() #it's also the order
for c in countries:
    columns.append(c)
heatDf = pd.DataFrame(columns = columns)
l = [None] * len(columns)
for trained in countries:
    test = temp1[temp1["trainedOn"] == trained]
    df2 = pd.DataFrame([l],columns=columns).fillna(0.0)
    df2["trainedOn"] = trained
    for idx,row in test.iterrows():
        df2[row["testedOn"]] = row["f1"]
    heatDf = heatDf.append(df2)
heatDf.reset_index(drop=True,inplace=True)

#plot heatmap
Index= countries
Cols = countries
df = pd.DataFrame(heatDf.drop("trainedOn",axis=1).transpose().values, index=Index, columns=Cols)
fig, ax = plt.subplots(figsize = (14, 9))

sns.heatmap(df, annot=True,cmap="viridis",vmin=0, vmax=0.75) #to have the same colouring than before
plt.title("F1 score heatmap (Dummy Classifier)")
plt.ylabel("Countries used as test set")
plt.xlabel("Countries used as training set")
plt.xticks(rotation=40)
plt.show()
print(bins)

# train on one, test on one

In [ ]:
def trainByCountry2(data,countries):
    #for every country, use it as test set
    nbins  = 3
    labels=["low","medium","high"]
    wine2 = pd.DataFrame()
    wine2 = data
    g,b = pd.qcut(data["points"],nbins,labels=labels,retbins=True)   #uses quartiles and statistic stuff
    g.tolist()
    wine3 = wine2.drop(["points"],axis=1)
    wine3["category"] = g
    df = pd.DataFrame()
    for el in countries:
        res = applyTest2(wine3,el)
        df = df.append(res)
    return df

def applyTest2(data,objCountry):
  
    #set many different combinations
    #only on these
    allfeatures = []
    args = []
    for el in wine_base.columns:
        if el != "description":
            allfeatures.append(el)   
        if el != "description" and el != "points" and el != "country":
            args.append(el)
    #just train on price
    #args = ["price"]
    combos = {"depth":[6],"args":[args]}
    #5,6,7 sono lo stesso
    r = buildCountryResult2(data,args,allfeatures,combos,objCountry)
    return r

def buildCountryResult2(data,args,allfeatures,combination,objCountry):
    cols = ["algorithm","ObjectiveCountry","input","precision","accuracy","f1","depth","combination","trainedOn","testedOn"]
    algorithm = "decTree"
    for lab in labels:
        cols.append(lab +"_prec")
        cols.append(lab +"_recall")
        cols.append(lab +"_f1")
        cols.append(lab +"_support")
    for el in args: #controlla che allfeats vada bene
        c = "feat_"+el
        if c not in cols:
            cols.append(c)
    results = pd.DataFrame()
    comb = 0
    row = 0
    for el in combination["args"]: 
        countries = data["country"].unique().tolist()
        interestingC =[]
        for gg in countries:
            if gg != objCountry:
                interestingC.append(gg)
      #  countries = countries.remove(objCountry) 
        el = el + ["country","category"]
        X = data[el]
        #create test and training sets
        X_train = X[X["country"] == objCountry]
        y_train = pd.DataFrame()
        y_train = list(X_train["category"])
        X_train.drop(["country","category"],axis=1,inplace=True)

        for countr in interestingC:
            X_test = X[X["country"] == countr]
            y_test = X_test["category"]
            X_test.drop(["country","category"],axis=1,inplace=True)
            for depth in combination["depth"]:
                    row = row + 1  
                    #train stuff
                    classifier = DecisionTreeClassifier(max_depth=depth)
                    classifier.fit(X_train,y_train)
                    y_pred = classifier.predict(X_test)  
                    acc = accuracy_score(y_test,y_pred)#average="macro")
                    weightedPrec = precision_score(y_test,y_pred,average="weighted")
                    f1 = f1_score(y_test,y_pred,average="weighted")
                    nations = objCountry +"->"+countr
                    rowData = [algorithm,objCountry,el,weightedPrec,acc,f1,depth,nations,objCountry,countr]
                    precision,recall,fscore,support = precision_recall_fscore_support(y_test, y_pred)
                    for i in range(0,len(labels)):
                        rowData.append(precision[i])
                        rowData.append(recall[i])
                        rowData.append(fscore[i])
                        rowData.append(support[i])
                        temp = {}
                    c = zip(el,classifier.feature_importances_)
                    names = []
                    for name,importance in c:
                        temp[name] = importance
                        names.append(name)
                    for feat in names:
                        #if feat not in allfeatures:
                         #   rowData.append(100) #100 is an impossible value not to be taken into account
                        #else:
                            rowData.append(temp[feat]) #qualcosa non quadra con le liste di input e colonne per appendere i dataframe con le relative feature importances
                    df2 = pd.DataFrame([rowData],columns=cols)
                    results = results.append(df2,ignore_index=True)
            comb = comb + 1
    return results

In [ ]:
twoComb = trainByCountry2(wine_base,countries)
twoComb.drop("input",inplace=True,axis=1)
twoComb

In [ ]:
temp1 = pd.DataFrame()
feats=[]
for f in twoComb.columns:
    if f != "algorithm" and f != "ObjectiveCountry" and f != "depth":
        feats.append(f)
for f in feats:
    temp1[f] = twoComb[f]
temp1 = temp1.groupby("combination")
t2 =pd.DataFrame()
for col in feats:
    if col != "combination" and col != "trainedOn" and col != "testedOn":
        t2[col] = temp1[col].mean()

t2.reset_index(inplace=True)
t2

In [ ]:
temp = t2
temp = temp.sort_values(["f1"],ascending=False)
temp = temp.head(10)
for idx,row in temp.iterrows():
    both = []
    for col in temp.columns:
        if row[col] != 0 and col != "combination" and col.startswith("feat"): #THIS TAKES INTO ACCOUNT EVEN PRICE WHICH IS A LOT LESS IMPORTANT COMPARED TO THE FIRST APPROACH
            both.append([col,row[col]])
    
    from operator import itemgetter
    both = sorted(both, key=itemgetter(1))
    
    toPlot = []
    values = []    
    for el in both:
        toPlot.append(el[0])
        values.append(el[1])
    
    # width of the bars
    barWidth = 0.5
    
    # Choose the height of the blue bars
    bars1 = values

    # The x position of bars
    r1 = np.arange(len(toPlot))
    fig, ax = plt.subplots(figsize = (10, 7))

    # Create blue bars
    plt.barh(r1, bars1, height=barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7)
    #plt.bar(r1,bars1,width = barWidth..)
    ys = toPlot
    # general layout
    #plt.xticks([r for r in range(len(bars1))], ys,rotation='vertical')
    #plt.yticks([r for r in range(len(bars1))], ys,rotation='vertical')
    plt.yticks([r for r in range(len(bars1))], ys)
    plt.title("Country combination: "+ row["combination"])
    plt.ylabel('Features')
    plt.xlabel("Importance")
    # Show graphic
    plt.show()

(media fra i risultati e li normalizzi) divide every mean obtained on germany by the mean of germany. Look at the average f1 by target country, if it's higher than 64/65, indaga perchè è più alto del 65 ottenuto con 70% train 30% test

In [ ]:
#Nation to nation heatmap

temp1 = pd.DataFrame()
feats=["trainedOn","testedOn","f1"]
for f in feats:
    temp1[f] = twoComb[f]
temp1 = temp1.groupby(["trainedOn","testedOn"])
t2 =pd.DataFrame()
for col in feats:
    if col != "combination" and col != "trainedOn" and col != "testedOn":
        t2[col] = temp1[col].mean()
t2.reset_index(inplace=True)
#t2.sort_values("f1",ascending=False)
columns = ["trainedOn"]
countries = t2["trainedOn"].unique().tolist() #it's also the order
for c in countries:
    columns.append(c)
heatDf = pd.DataFrame(columns = columns)
l = [None] * len(columns)
for trained in countries:
    test = t2[t2["trainedOn"] == trained]
    df2 = pd.DataFrame([l],columns=columns).fillna(0.0)
    df2["trainedOn"] = trained
    for idx,row in test.iterrows():
        df2[row["testedOn"]] = row["f1"]
    heatDf = heatDf.append(df2)
heatDf.reset_index(drop=True,inplace=True)

#plot heatmap
Index= countries
Cols = countries
df = pd.DataFrame(heatDf.drop("trainedOn",axis=1).transpose().values, index=Index, columns=Cols)
fig, ax = plt.subplots(figsize = (14, 9))

sns.heatmap(df, annot=True,cmap="viridis")
plt.title("F1 score heatmap (Decision Tree of depth 6)")
plt.ylabel("Countries used as test set")
plt.xlabel("Countries used as training set")
plt.xticks(rotation=40)
plt.show()
print(bins)

In [ ]:
heatDf

Prendi i risultati del dummy classifier e comparali con questi, magari mostra la differenza

In [ ]:
for c in countries:
    ada = wine_base[wine_base["country"] == c]
    
    if (87 <= ada["points"].mean() <= 90 ):
        print(c + " has the mean points value in the medium bin")
    else: 
        print(c + " not in the medium " + str(ada["points"].mean()))

if you plot the heatmap based only on price, you see the results are worse and slightly worse. If the training set's points distribution is in the medium bin, values will be higher and if the distributions are similar, the result will be better.
If the distribution is not in the medium bin but the pts distr is similar (austria-> germany) you'll get a high value?


by using only price, austria sucks across the board, there you really can see the improvement in the prediction with the other features

In [ ]:
meanTrainF1 = twoComb.groupby("trainedOn")["f1"].mean()
meanTrainF1
#in this case, i divide Argentina (train) -> Spain (test)'s F1 by the mean F1 obtained from Argentina -> everything else
#if it's more than one, than the results on that country are above the average
countries = t2["trainedOn"].unique().tolist()
for trainedOn in countries:
    data = t2[t2["trainedOn"] == trainedOn]
    both = []
    for idx,row in data.iterrows():
        both.append([row["testedOn"], row["f1"]/meanTrainF1[trainedOn]])
        #print(toPlot)
        #print(values)

    from operator import itemgetter
    both = sorted(both, key=itemgetter(1))
    print(both)

    toPlot = []
    values = []    
    for el in both:
            toPlot.append(el[0])
            values.append(el[1])

            # width of the bars
    barWidth = 0.5

            # Choose the height of the blue bars
    bars1 = values

            # The x position of bars
    r1 = np.arange(len(toPlot))
    fig, ax = plt.subplots(figsize = (10, 7))

            # Create blue bars
    plt.barh(r1, bars1, height=barWidth, color = "#a6cee3", edgecolor = 'black', capsize=7)
           #plt.bar(r1,bars1,width = barWidth..)
    ys = toPlot
            # general layout
            #plt.xticks([r for r in range(len(bars1))], ys,rotation='vertical')
            #plt.yticks([r for r in range(len(bars1))], ys,rotation='vertical')
    plt.yticks([r for r in range(len(bars1))], ys)
    plt.title("Country on which the model is trained: " +trainedOn)
    plt.ylabel('Country on which the model is tested')
    plt.xlabel("F1(country)/ MeanF1(" + trainedOn+")")
            # Show graphic
    plt.show()

In [ ]:
meanTrainF1 = twoComb.groupby("trainedOn")["f1"].mean()
meanTrainF1


In [ ]:
fig, ax = plt.subplots(figsize = (14, 5))
ax.set(xticks=wine_base["points"].unique())
temp = wine_base[(wine_base["country"] == "Austria")]
sns.distplot(temp["points"],label="austria",bins = 20,hist=False,norm_hist=False,kde=True)
print("austria mean points ",temp["points"].mean())
temp = wine_base[(wine_base["country"] == "Australia")]
sns.distplot(temp["points"],label="australia",bins = 20,hist=False,norm_hist=False,kde=True)
print("australia mean points",temp["points"].mean())
print(bins)

if u look at the plot below, no difference between chile/argentina and argentina/chile because points distributions are very similar

it seems that the closest the training set to the mean points in the dataset, the better result on any test set

chile's mean is not in the medium, so this is pretty much confirmed (italyy is in the medium):

there is a problem: if you train on chile and test on italy 56, if you train on italy and test on chile 69.

Check the medium bin f1

In [ ]:
temp1 = pd.DataFrame()
feats=["trainedOn","testedOn","medium_f1"]
for f in feats:
    temp1[f] = twoComb[f]
temp1 = temp1.groupby(["trainedOn","testedOn"])
t2 =pd.DataFrame()
for col in feats:
    if col != "combination" and col != "trainedOn" and col != "testedOn":
        t2[col] = temp1[col].mean()
t2.reset_index(inplace=True)
#t2.sort_values("f1",ascending=False)
columns = ["trainedOn"]
countries = t2["trainedOn"].unique().tolist() #it's also the order
for c in countries:
    columns.append(c)
heatDf = pd.DataFrame(columns = columns)
l = [None] * len(columns)
for trained in countries:
    test = t2[t2["trainedOn"] == trained]
    df2 = pd.DataFrame([l],columns=columns).fillna(0.0)
    df2["trainedOn"] = trained
    for idx,row in test.iterrows():
        df2[row["testedOn"]] = row["medium_f1"]
    heatDf = heatDf.append(df2)
heatDf.reset_index(drop=True,inplace=True)

#plot heatmap
Index= countries
Cols = countries
df = pd.DataFrame(heatDf.drop("trainedOn",axis=1).transpose().values, index=Index, columns=Cols)
fig, ax = plt.subplots(figsize = (14, 9))

sns.heatmap(df, annot=True,cmap="viridis")
plt.title("F1 (medium bin) score heatmap")
plt.ylabel("Countries used as test set")
plt.xlabel("Countries used as training set")
plt.xticks(rotation=40)
plt.show()


In [ ]:
t2

# check why chile some countries are easier to predict 

In [ ]:
ofInterest = ["Chile","Argentina","Austria","Germany"]
for t1 in countries:
    for t in ofInterest:
        if t != t1:
            print("--------RESULTS OF-------"+ t + " " + t1 + ": (trained/tested) f1 ")
            row = t2[(t2["trainedOn"]==t) & (t2["testedOn"]==t1)]
            print(row)
            row1 = t2[(t2["trainedOn"]==t1) & (t2["testedOn"]==t)]
            print(row1)
            fig, ax = plt.subplots(figsize = (14, 5))
            temp = wine_base[(wine_base["country"] == t) & (wine_base["price"] < 200)]
            temp1 = wine_base[(wine_base["country"] == t1) & (wine_base["price"] < 200)]
            plt.subplot(1, 2, 1)
            plt.title("Points distribution for " + t + " and " + t1 )
            sns.distplot(temp1["points"],label=t1,bins = 20,hist=False,norm_hist=True,kde=True)
            sns.distplot(temp["points"],label=t,bins = 20,hist=False,norm_hist=True,kde=True)
            plt.legend()
            #plt.subplot(2,2,2)
            plt.subplot(1, 2, 2)
            plt.title("Price distribution for " + t + " and " + t1)
            sns.distplot(temp1["price"],label=t1,hist=False,norm_hist=True,kde=True)
            sns.distplot(temp["price"],label=t,hist=False,norm_hist=True,kde=True)
            plt.legend()
            #plt.subplot(2, 2,3)
            #plt.title("word_count distribution for " + t + " and " + t1)
            #sns.distplot(temp1["word_count"],label=t1,bins = 20,hist=False,norm_hist=True,kde=True)
            #sns.distplot(temp["word_count"],bins = 20,label=t,hist=False,norm_hist=True,kde=True)
            #plt.legend()
            #plt.subplot(2, 2,4)
            #plt.title("word2vec similarity distribution for " + t + " and " + t1)
            #sns.distplot(temp1["similarityTop3WinesByVariety"],label=t1,bins = 20,hist=False,norm_hist=True,kde=True)
            #sns.distplot(temp["similarityTop3WinesByVariety"],bins = 20,label=t,hist=False,norm_hist=True,kde=True)
            #plt.legend()
            plt.show()


# PRINT DECISION TREE TRAINED ON USA

In [ ]:
def getUSDecTree(data,countries):
    #for every country, use it as test set
    nbins  = 3
    labels=["low","medium","high"]
    wine2 = pd.DataFrame()
    wine2 = data
    g,b = pd.qcut(data["points"],nbins,labels=labels,retbins=True)   #uses quartiles and statistic stuff
    g.tolist()
    wine3 = wine2.drop(["points"],axis=1)
    wine3["category"] = g
    df = pd.DataFrame()
    for el in countries:
        res, cols = applyTest3(wine3,el)
    return res,cols

def applyTest3(data,objCountry):
  
    #set many different combinations
    #only on these
    allfeatures = []
    args = []
    for el in wine_base.columns:
        if el != "description":
            allfeatures.append(el)   
        if el != "description" and el != "points" and el != "country":
            args.append(el)
    #just train on price
    #args = ["price"]
    combos = {"depth":[6],"args":[args]}
    #
    r, cols = buildCountryResult3(data,args,allfeatures,combos,objCountry)
    return r, cols 

def buildCountryResult3(data,args,allfeatures,combination,objCountry):
    cols = ["algorithm","ObjectiveCountry","input","precision","accuracy","f1","depth","combination","trainedOn","testedOn"]
    algorithm = "decTree"
    for lab in labels:
        cols.append(lab +"_prec")
        cols.append(lab +"_recall")
        cols.append(lab +"_f1")
        cols.append(lab +"_support")
    for el in args: #controlla che allfeats vada bene
        c = "feat_"+el
        if c not in cols:
            cols.append(c)
    results = pd.DataFrame()
    comb = 0
    row = 0
    for el in combination["args"]: 
        countries = data["country"].unique().tolist()
        interestingC =[]
        for gg in countries:
            if gg != objCountry:
                interestingC.append(gg)
      #  countries = countries.remove(objCountry) 
        el = el + ["country","category"]
        X = data[el]
        #create test and training sets
        X_train = X[X["country"] == "US"]  #prendi solo US
        y_train = pd.DataFrame()
        y_train = list(X_train["category"])
        X_train.drop(["country","category"],axis=1,inplace=True)

        for countr in interestingC:
            X_test = X[X["country"] == countr]
            y_test = X_test["category"]
            X_test.drop(["country","category"],axis=1,inplace=True)
            for depth in combination["depth"]:
                    row = row + 1  
                    #train stuff
                    classifier = DecisionTreeClassifier(max_depth=depth)
                    classifier.fit(X_train,y_train)
                    y_pred = classifier.predict(X_test)  
                    precision,recall,fscore,support = precision_recall_fscore_support(y_test, y_pred)
                    c = zip(el,classifier.feature_importances_)
                    names = []
                    for name,importance in c:
                        temp[name] = importance
                        names.append(name)

    return classifier, names

In [ ]:
clf, cols = getUSDecTree(wine_base,["Australia"])
import os
import sys
import pydotplus 
from sklearn import tree
from IPython.display import Image  
def conda_fix(graph):
    path = os.path.join(sys.base_exec_prefix, "Library", "bin", "graphviz")
    paths = ("dot", "twopi", "neato", "circo", "fdp")
    paths = {p: os.path.join(path, "{}.exe".format(p)) for p in paths}
    graph.set_graphviz_executables(paths)
                    
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=cols , class_names=labels, filled=True, rounded=True, special_characters=True,max_depth=3)  
graph = pydotplus.graph_from_dot_data(dot_data)  
conda_fix(graph)
Image(graph.create_png())               
